In [1]:
import numpy as np
import pandas as pd
%matplotlib notebook

In [2]:
n = str(1) #chiller number
datatypes = ["Power","ConFlow","EvaFlow","Temp"]
df={}
for datatype in datatypes:
    df[datatype] = pd.read_csv("data/Chiller"+n+"_"+datatype+"_full.csv")
    df[datatype]['ts']=pd.to_datetime(df[datatype]['ts'])
    df[datatype]['ts']=df[datatype]['ts'].dt.floor('min')
    print(datatype)
    print(df[datatype].columns)
    
ext_df_may = pd.read_csv("data/ext/NUS May OAT.csv")
ext_df_jun = pd.read_csv("data/ext/NUS June OAT.csv")
ext_df_july = pd.read_csv("data/ext/NUS July OAT.csv")
ext_df_aug = pd.read_csv("data/ext/NUS August OAT.csv")
ext_df_sept = pd.read_csv("data/ext/NUS September OAT.csv")
ext_df_oct = pd.read_csv("data/ext/NUS October OAT.csv")
ext_df_nov = pd.read_csv("data/ext/NUS November OAT.csv")
ext_df=pd.concat([ext_df_may, ext_df_jun, ext_df_july, ext_df_aug, ext_df_sept, ext_df_oct,ext_df_nov])

ext_df['ts']=pd.to_datetime(ext_df['ts'])
ext_df['ts']=ext_df['ts'].dt.floor('min')
ext_df=ext_df[['ts','value']]

Power
Index(['ts', 'gatewayId', 'linkQuality', 'ch1Watt', 'ch2Watt', 'ch3Watt',
       'totalPositiveWattHour', 'totalNegativeWattHour', 'ch1Current',
       'ch2Current', 'ch3Current', 'ch1Voltage', 'ch2Voltage', 'ch3Voltage',
       'ch1PowerFactor', 'ch2PowerFactor', 'ch3PowerFactor', 'voltage12',
       'voltage23', 'voltage31', 'ch1Hz', 'ch2Hz', 'ch3Hz', 'i1THD', 'i2THD',
       'i3THD', 'v1THD', 'v2THD', 'v3THD'],
      dtype='object')
ConFlow
Index(['ts', 'gatewayId', 'linkQuality', 'flowRate', 'flowSpeed',
       'totalFlowRate', 'positiveTotalFlow', 'positiveTotalFlowDecimal'],
      dtype='object')
EvaFlow
Index(['ts', 'gatewayId', 'linkQuality', 'flowRate', 'flowSpeed',
       'totalFlowRate', 'positiveTotalFlow', 'positiveTotalFlowDecimal'],
      dtype='object')
Temp
Index(['ts', 'gatewayId', 'linkQuality', 'ain1', 'ain2', 'ain3', 'ain4',
       'ain5', 'voltage1', 'voltage2', 'voltage3', 'voltage4', 'voltage5',
       'value1', 'value2', 'value3', 'value4'],
      dtype='

In [3]:
collist={}
collist['Power'] = ['ch'+str(i+1)+'Watt' for i in range(3)]
collist['Temp'] = ['value'+str(i+1) for i in range(4)]
collist['ConFlow'] = ['flowRate', 'flowSpeed']
collist['EvaFlow'] = ['flowRate', 'flowSpeed']

truncate_df = {}
for key in df.keys():
    truncate_df[key] = (df[key][['ts']+collist[key]])
    print(key)
    print(len(truncate_df[key].ts))
    print(len(truncate_df[key].ts.unique()))
    truncate_df[key]=truncate_df[key].groupby('ts', as_index=False).mean()

truncate_df['ConFlow'].columns = ['ts', 'conflowRate', 'conflowSpeed']
truncate_df['EvaFlow'].columns = ['ts', 'evaflowRate', 'evaflowSpeed']

EvaFlow
293430
281505
Temp
301943
288724
ConFlow
293416
281396
Power
295954
283177


In [5]:
print("extTemp")
print(len(ext_df.ts))
print(len(ext_df.ts.unique()))
ext_df=ext_df.groupby('ts', as_index=False).mean()

extTemp
254212
253724


In [13]:
PowerTemp = pd.DataFrame.merge(truncate_df['Power'], truncate_df['Temp'])
Flows = pd.DataFrame.merge(truncate_df['ConFlow'], truncate_df['EvaFlow'])
fullDF = pd.DataFrame.merge(PowerTemp,Flows)

In [ ]:
fullDF_wext = pd.DataFrame.merge(fullDF, ext_df)
fullDF_wext.index=fullDF_wext.ts
fullDF_wext.drop('ts', axis=1, inplace=True)
fullDF_wext.to_csv("data/Chiller"+n+"_full_ext.csv")

In [ ]:
fullDF.index=fullDF.ts
fullDF.drop('ts', axis=1, inplace=True)
fullDF.head()
fullDF.to_csv("data/Chiller"+n+"_full.csv")